In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cp -av /content/drive/MyDrive/rec/items.csv items.csv
%cp -av /content/drive/MyDrive/rec/ratings.csv ratings.csv

'/content/drive/MyDrive/rec/items.csv' -> 'items.csv'
'/content/drive/MyDrive/rec/ratings.csv' -> 'ratings.csv'


In [3]:
!pip install implicit

     |████████████████████████████████| 1.1MB 30.2MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406415 sha256=b1611af04d25bddebada126546568ab68855d02de63e1994d1217f19ec3dfe2b
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import pickle

from implicit.evaluation import train_test_split, precision_at_k, mean_average_precision_at_k
import implicit
import seaborn as sns
movies = pd.read_csv("items.csv")
ratings = pd.read_csv("ratings.csv")

In [5]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
alpha = 40

user_item = csr_matrix( ([alpha]*len(ratings['movie_id']), (ratings['user_id'], ratings['movie_id']) ))
item_user = user_item.T.tocsr()

In [13]:
train, test = train_test_split(item_user, train_percentage=0.8)

model = implicit.als.AlternatingLeastSquares(factors=100, 
    regularization=0.1, iterations=20, calculate_training_loss=False)
model.fit(train)

with open('model_collaborative.sav', 'wb') as pickle_out:
    pickle.dump(model, pickle_out)

p_at_k = precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10)
p_at_k

(1683, 944)


0.27697201017811707

In [9]:
 user = 1
 recommended, _ =  zip(*model.recommend(user-1, user_item))

In [10]:
from scipy import sparse

sparse.save_npz("collaborative.npz", user_item)

In [11]:
recommended

(557, 559, 556, 553, 558, 563, 560, 561, 562, 1682)

In [12]:
recommend_frame = []
for val in recommended:
    movie_idx = ratings.iloc[val]['movie_id']
    idx = movies[movies['movie_id'] == movie_idx].index
    recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0]})
df = pd.DataFrame(recommend_frame,index=range(1,11))
df

,Title
1,Kundun (1997)
2,Rear Window (1954)
3,"Omen, The (1976)"
4,Kull the Conqueror (1997)
5,With Honors (1994)
6,"Birdcage, The (1996)"
7,Great Expectations (1998)
8,"Hunt for Red October, The (1990)"
9,Evita (1996)
10,"House of Yes, The (1997)"
